In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_
from data_utils import Dictionary, Corpus

# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters
embed_size = 128
hidden_size = 1024
batch_size = 20
num_layers = 1
num_epochs = 5
num_samples = 1000  # Generate a text file with 1000 words
seq_length = 30
learning_rate = 0.002

# Load 'Penn Treebank' dataset
corpus = Corpus()
ids = corpus.get_data('./data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length  # size(1)???

# RNN based on language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)
        
        # Foward pass
        out, (h, c) = self.lstm(x, h)
        
        # Reshape output to (batch_size*seq_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)
    
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [3]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states]

# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell state
    states = [torch.zeros(num_layers, batch_size, hidden_size).to(device), 
              torch.zeros(num_layers, batch_size, hidden_size).to(device)]
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Every seq_length is a mini-batch
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)  # pass to next layer
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))            

Epoch [1/5], Step[0/1549], Loss: 9.2123, Perplexity: 10020.00
Epoch [1/5], Step[100/1549], Loss: 6.0187, Perplexity: 411.06
Epoch [1/5], Step[200/1549], Loss: 5.9184, Perplexity: 371.81
Epoch [1/5], Step[300/1549], Loss: 5.7547, Perplexity: 315.68
Epoch [1/5], Step[400/1549], Loss: 5.6577, Perplexity: 286.50
Epoch [1/5], Step[500/1549], Loss: 5.1535, Perplexity: 173.04
Epoch [1/5], Step[600/1549], Loss: 5.2044, Perplexity: 182.06
Epoch [1/5], Step[700/1549], Loss: 5.3305, Perplexity: 206.55
Epoch [1/5], Step[800/1549], Loss: 5.1673, Perplexity: 175.45
Epoch [1/5], Step[900/1549], Loss: 5.0541, Perplexity: 156.66
Epoch [1/5], Step[1000/1549], Loss: 5.1662, Perplexity: 175.24
Epoch [1/5], Step[1100/1549], Loss: 5.3374, Perplexity: 207.96
Epoch [1/5], Step[1200/1549], Loss: 5.1793, Perplexity: 177.57
Epoch [1/5], Step[1300/1549], Loss: 5.1110, Perplexity: 165.84
Epoch [1/5], Step[1400/1549], Loss: 4.8451, Perplexity: 127.11
Epoch [1/5], Step[1500/1549], Loss: 5.1596, Perplexity: 174.10
Ep

In [4]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set initial hidden state and cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))
        
        # Select one word id randomly
        prob = torch.ones(vocab_size)
        inputs = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)
        
        for i in range(num_samples):
            output, state = model(inputs, state)
            
            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            
            # Fill input with sampled word id for the next time step
            inputs.fill_(word_id)
            
            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)
            
            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

# Save the model checkpoints
torch.save(model.state_dict(), 'model.ckpt')

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt
